In [1]:
# import pyspark
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
# from pyspark.sql.window import *
# from pyspark.sql.types import ArrayType, FloatType, LongType, IntegerType

# from pyspark.sql.window import Window
# from pyspark.sql.functions import row_number

import numpy as np
import pandas as pd
import os
import gc

In [2]:
## get all valid sessions with orders actions
## there are 150179 / 1783737 sessions have orders actions, 301057 / 1783737 have carts, 1737968 / 1783737 have clicks
valid_order_sessions = pd.read_json("../../allData/validationData/out_7day_test/test_labels.jsonl", lines=True)
valid_order_sessions['aids'] = valid_order_sessions["labels"].apply(lambda x: x.get("orders"))
valid_order_sessions = valid_order_sessions.loc[valid_order_sessions.aids.notnull()]
valid_order_sessions = valid_order_sessions.drop("labels", axis = 1)
valid_order_sessions = valid_order_sessions.drop("aids", axis = 1)
gc.collect()

## inner join order_features and valid_order_sessions, get all features of valid session rec features
## 13601844 recommendations are valid. 
order_features_raw = pd.read_parquet("../../allData/features/orders_features_100_per_session.parquet")
order_features_valid_session = pd.merge(order_features_raw, valid_order_sessions, on="session")

del valid_order_sessions, order_features_raw
gc.collect()

0

In [15]:
## Exploding the 150179 sessions with orders actions, there are 311649 unique order activities. 
orders_labels = pd.read_json("../../allData/validationData/out_7day_test/test_labels.jsonl", lines=True)
orders_labels['aids'] = orders_labels["labels"].apply(lambda x: x.get("orders"))
orders_labels = orders_labels[orders_labels.aids.notnull()]
orders_labels = orders_labels.drop("labels", axis = 1)
# orders_labels["gt_type"] = 2
orders_labels = orders_labels.set_index(['session']).apply(pd.Series.explode).reset_index()
orders_labels["gt"] = 1
gc.collect()

0

In [22]:
## Joining all features with labels, 218522 / 311659 were recalled by querying top 100 in the CF systems. 
# 218522 / 311659 =>  0.7011573546728957 translate the max possible we can get in order sector. 
orders_full_df = pd.merge(order_features_valid_session, orders_labels, on=["session", "aids"], how='left')

In [24]:
## output this file for LightGBM ranker to learn 
orders_full_df.head(2)

,session,aids,prev_int,seq_w,time_w,ops_total,session_len,session_unique_aid,rank_score,gt
0,11098528,11830,True,0.231144,3.0,0,1,1,13.868665,1.0
1,11098528,1732105,False,0.231144,3.0,0,1,1,1.386866,NaN


In [25]:
orders_full_df.to_parquet("../../allData/features/ready_GBM_orders_100_features_V2.parquet")